In [ ]:
import requests
import polars as pl
import json
import os
from datetime import datetime,timezone


pl.Config.set_tbl_rows(10)
pl.Config.set_tbl_cols(100)

In [ ]:
ctt = pl.read_csv("coins_to_track.csv")
','.join(ctt['Symbol'])

In [3]:
cmc_key = "57eccd1c-b1de-4cff-8b6a-b4206ffded67"

headers = {
    "X-CMC_PRO_API_KEY": cmc_key,
    "Content-Type": "application/json"
}

parameters = {
    # "symbol": ','.join(ctt['Symbol']),
    "symbol": 'BTC',
    "skip_invalid": "false"
}

# session = requests.Session()
# session.headers.update(headers)

x = json.loads(session.get(f"https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest", params = parameters).text)

In [300]:
def cmc_extract(server_url: str, headers: dict, parameters: dict):

    """
    returns: api response generator 
    """

    # coins_to_track = pl.read_csv("coins_to_track.csv")
    # symbols = ','.join(ctt['Symbol'])

    # parameters.update(symbol=symbols)
    session = requests.Session()
    session.headers.update(headers)

    responses = json.loads(session.get(server_url, params = parameters).text)

    return responses




In [ ]:
coins_to_track = pl.read_csv("coins_to_track.csv")
symbols = ','.join(coins_to_track['Symbol'][:])
parameters = {}
parameters.update(symbol=symbols)

print(parameters)

# QUOTES LATEST

In [ ]:
# coins_to_track = pl.read_csv("coins_to_track.csv")
# symbols = ','.join(ctt['Symbol'][:])
# parameters = {}
# parameters.update(symbol=symbols)

# print(parameters)

responses = cmc_extract(server_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/quotes/latest", headers = headers, parameters = parameters)

returned_data = responses["data"]
returned_symbols = list(returned_data.keys())

quote_dfs = [pl.DataFrame(returned_data[symbol]["quote"]["USD"]).with_columns(pl.lit(symbol).alias("symbol"), pl.lit(int(returned_data[symbol]["cmc_rank"])<=10).alias("IsTopCurrency")) for symbol in returned_symbols]
df = pl.concat(quote_dfs, how = 'vertical_relaxed')

ts = datetime.now(timezone.utc)
df = df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))
df.write_csv(file = fr"src/extracts/{ts.strftime('%Y%m%dT%H%M%S')}_quotes.csv")


# METADATA

In [475]:
responses2 = cmc_extract(server_url = "https://pro-api.coinmarketcap.com/v2/cryptocurrency/info", headers = headers, parameters = {"symbol":symbols, "skip_invalid": "true"})

returned_data2 = responses2["data"]
returned_symbols2 = list(returned_data2.keys())

In [ ]:
meta_dfs = [pl.from_records(returned_data2[symbol]) for symbol in returned_data2.keys()]
meta_df = pl.concat([df for df in meta_dfs if not df.is_empty()], how = "vertical_relaxed")

platform = meta_df.select(
    # pl.col("urls").struct.unnest()
    pl.col("platform").struct.unnest()
    # pl.col("contract_address").list.json_encode()
    )
platform = platform.rename({i:f"platform_{i}" for i in platform.columns})

urls = meta_df.select(
    # pl.col("urls").struct.unnest()
    pl.col("urls").struct.unnest()
    # pl.col("contract_address").list.json_encode()
    )

In [ ]:
contract_address = meta_df.select(
    pl.col("contract_address").list.first().struct.unnest()
    )

# contract_address = contract_address.select(
#     pl.col("contract_address"),
#     pl.col("platform").struct.unnest()
# )

# contract_address = contract_address.select(
#     # pl.col("contract_address"),
#     # pl.col("name")
#     pl.col("coin").struct.unnest()
# )

# contract_address = contract_address.rename({i:f"contract_address_{i}" for i in contract_address.columns})
contract_address = contract_address.select(
    pl.col("contract_address"),
    pl.col("platform").struct.json_encode()
    )

contract_address

In [ ]:
contract_address = meta_df.with_columns(
    pl.col("contract_address").list.first().struct.unnest()
    )
contract_address

In [ ]:
dict(zip(meta_df.columns,meta_df.dtypes))

In [ ]:
meta_df.select(
    pl.col("contract_address").list.first().struct.unnest()
    )

# Mapping Data

In [367]:
responses = cmc_extract(server_url = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/map", headers = headers, parameters = {})

returned_data = responses["data"]
# returned_symbols = list(returned_data.keys())

map_dfs = [pl.from_records(returned_data).sort(by = 'symbol').filter(pl.col('symbol').eq(symbol)) for symbol in coins_to_track['Symbol']]

map_df = pl.concat(map_dfs, how = 'vertical_relaxed')
map_df = map_df.with_columns((pl.col("rank").cast(pl.Int64) <= 10).alias("IsTopCurrency"))

ts = datetime.now(timezone.utc)
map_df = map_df.with_columns(pl.lit(ts.strftime("%Y-%m-%dT%H:%M:%SZ")).alias("LoadedWhen"))
map_df.select(
    "name",
    "symbol",
    "slug",
    "is_active",
    "status",
    "first_historical_data",
    "last_historical_data",
    "IsTopCurrency"
    ).write_csv(file = fr"src/extracts/map/{ts.strftime('%Y%m%dT%H%M%S')}_map.csv")